In [1]:
%matplotlib notebook

import time
import sys
import numpy as np
from collections import OrderedDict

# import sys

paths = [
# #     r"C:\Users\Administrator\Code\qcodes_wp",
    r"C:\Users\Administrator\Code\qcodes_jens",
    r"C:\Users\Administrator\Code\pysweep2.0",
]

for p in paths:
    if p not in sys.path:
        sys.path.append(p)

import qcodes
from qcodes.instrument_drivers.QuTech.IVVI import IVVI
from qcodes.instrument_drivers.stanford_research.SR865 import SR865
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A


from pysweep import measurement
from pysweep import sweep, Measurement
from pysweep import data_plot

## Initialization

In [2]:
#Measurement.use_storage("json")
Measurement.use_storage("np")

In [3]:
ivvi = IVVI("ivvi", "ASRL5::INSTR")
sr1 = SR865("sr1", "GPIB0::3::INSTR")
key = Keysight_34465A("key1", "USB0::0x2A8D::0x0101::MY57503596::INSTR")
# RF = RohdeSchwarz_SGS100A('RF', address="TCPIP0::169.254.2.20")
#key2 = Keysight_34465A("key2", "USB0::0x2A8D::0x0101::MY57503135::INSTR")

C:\Users\Administrator\Code\qcodes_jens\qcodes\instrument\parameter.py:182: UserWarning: Delay kwarg is deprecated. Replace with inter_delay or post_delay as needed
  warnings.warn("Delay kwarg is deprecated. Replace with "


Initialized IVVI-rack in 0.34s
Connected to: Stanford_Research_Systems SR860 (serial:003104, firmware:V1.47) in 0.10s
Connected to: Keysight Technologies 34465A (serial:MY57503596, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.16s


C:\Users\Administrator\Code\qcodes_jens\qcodes\instrument\parameter.py:207: UserWarning: Wrapping get method, original get method will not be directly accessible. It is recommended to define get_raw in your subclass instead.
  warnings.warn('Wrapping get method, original get method will not '


In [4]:
#Set some ramp rates for the DACs which aren't too painfully slow
for i in range(1,16):
    ivvi.parameters['dac{}'.format(i)].set_step(0.5)
    ivvi.parameters['dac{}'.format(i)].set_delay(0.001)

ivvi.dac1.set_delay(0.001)
ivvi.dac1.set_step(20)
ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(5)
ivvi.dac8.set_delay(0.001)
ivvi.dac8.set_step(5)
ivvi.dac12.set_delay(0.001)
ivvi.dac12.set_step(5)
ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(5)

C:\Users\Administrator\Code\qcodes_jens\qcodes\instrument\parameter.py:512: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
C:\Users\Administrator\Code\qcodes_jens\qcodes\instrument\parameter.py:524: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


In [9]:
ivvi.set_dacs_zero()

In [5]:
ivvi.close()

## Measurement definitions

In [5]:
#Set some ramp rates for the DACs which aren't too painfully slow
def setup(station, namespace): 
#    for i in range(1,16):
#        station.ivvi.parameters['dac{}'.format(i)].set_step(0.5)
#        station.ivvi.parameters['dac{}'.format(i)].set_delay(0.001)
        return {}

def cleanup(station, namespace): 
    return {}

#def measurement(station, namespace): 
#    
#    volt = station.key1.volt()
#    
#    return {
#        "measurement": {
#            "unit": " V", 
#            "value": 2*volt
#        }, 
#        "raw_reading": {
#            "unit": "V",
#            "value": volt
#        }
#    }

def dc_measurement(station, namespace):
    
    #if hasattr(namespace, 't0'):
        #print('time since last point: {:.3f}'.format((time.time()-namespace.t0)*1e3))
    namespace.t0 = time.time()
    
    S1_gain = 1 # for now 
    NW_gain = 1e7 # Gain of current amp in V/A
    V_range = 1e-3 #Gain factor of voltage source module in V/V
    
    NW_res = 11700 + NW_gain*1e-4+100
    
    t0 = time.time()
    k1 = station.key1.volt()
    t1 = time.time() - t0
    #print("voltage readout took {:.3f} ms".format(t1 * 1e3))
    
    k1_scaled = k1/NW_gain
    #k2 = station.key2.volt()
    #k2_scaled = k2/S1_gain
   
    t0 = time.time()
    lockin1_x = station.sr1.X()
    lockin1_p = station.sr1.P()
    t1 = time.time() - t0
    #print("lock-in readout took {:.3f} ms".format(t1 * 1e3))
    
    if lockin1_x == 0:
        processed1 = 1e99
    else:
        processed1 = station.sr1.amplitude()*V_range*0.01 / (lockin1_x/NW_gain)-NW_res
    
    return OrderedDict({
        "Current": {"unit": "A", "value": k1_scaled}, 
        "k1": {"unit": "V", "value": k1},
        #"k2 scaled": {"unit": "V", "value": k2_scaled}, 
        #"k2": {"unit": "V", "value": k2},
        "lockin1_x": {"unit": "V", "value": lockin1_x}, 
        "lockin1_p": {"unit": "deg", "value": lockin1_p}, 
        "processed1": {"unit": "Ohm", "value": processed1}, 
        "conductance": {"unit": "h/2e", "value": 12906/processed1}
    })

def key1_measurement(station, namespace):
    
    S1_gain = 1 # for now 
    NW_gain = 1e7 # Gain of current amp in V/A
    V_range = 1e-3 #Gain factor of voltage source module in V/V
    
    NW_res = 11700 + NW_gain*1e-4+100
    
    k1 = station.key1.volt()
    k1_scaled = k1/NW_gain
      
       
    return OrderedDict({
        "Current": {"unit": "A", "value": k1_scaled}, 
        "k1": {"unit": "V", "value": k1},
        
           })

def homodyne_measurement(station, namespace):
   
    k2 = station.key1.volt() 
      
       
    return OrderedDict({
        "RF": {"unit": "V", "value": k2}
        
           })

def log_time(station, namespace):
    #print("Current ms:", time.time()%1 * 1e3)
    return {}

def clr(station, namespace):
    print("")
    return {}

## Basic measurement loops

In [7]:
station = qcodes.Station(key, ivvi, sr1)
Measurement.set_station(station)

C:\Users\Administrator\Code\qcodes_jens\qcodes\instrument\parameter.py:614: UserWarning: set_validator is deprected use `inst.vals = MyValidator` instead
  "set_validator is deprected use `inst.vals = MyValidator` instead")


In [12]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.RF.frequency, np.linspace(50e6, 200e6, 251)),
        [
            dc_measurement,
#             log_time, 
            #clr#, 
            #measurement
        ]
    )
).run(max_buffer_size=2)

AttributeError: 'Station' object and its delegates have no attribute 'RF'

In [ ]:
# measure.output()

In [ ]:
#out = measure.output("Current")

In [ ]:
#data_plot.DataPlot(out)

In [ ]:
import time 

def sleep(tm): 
    def inner(station, namespace): 
        time.sleep(tm)
        return {}
    
    return inner 

In [9]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.ivvi.dac1, np.linspace(-600, 1400, 101)),
        sweep(station.ivvi.dac4, np.linspace(0, 50, 200)), 
        #sleep(0.001),
        dc_measurement
    )
).run()

In [ ]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.ivvi.dac1, np.linspace(-1500, 1800, 101)),
        sweep(station.ivvi.dac9, np.linspace(0, 50, 200)), 
        #sleep(0.001),
        key1_measurement
    )
).run(max_buffer_size=2)

In [ ]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.ivvi.dac1, np.linspace(-1500, 2000, 88)),
        sweep(station.ivvi.dac6, np.linspace(0, 400, 400)), 
        #sleep(0.001),
        dc_measurement
    )
).run(max_buffer_size=2)

In [ ]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.ivvi.dac1, np.linspace(-2000, 2000, 101)),
        sweep(station.ivvi.dac5, np.linspace(0, 200, 200)), 
        #sleep(0.001),
        key1_measurement
    )
).run(max_buffer_size=2)

In [ ]:
measure = Measurement(
    setup, 
    cleanup, 
    (
        sweep(station.ivvi.dac1, np.linspace(-1500, 2000, 201)),
        sweep(station.ivvi.dac12, np.linspace(-50, 50, 400)), 
        #sleep(0.001),
        dc_measurement
    )
).run(max_buffer_size=2)

In [ ]:
key.NPLC(0.2)

In [6]:
ivvi.dac1(2000)

In [ ]:
sr1.amplitude(.2)

In [ ]:
print((time.time() % 1) * 1e3)
print(key.volt())
print((time.time() % 1) * 1e3)


In [ ]:
4.4%1

In [6]:
RF.frequency()

150000000.0

In [7]:
key.volt()

-0.000721189177